<a href="https://colab.research.google.com/github/ritzdevp/team-samosa-tvqa/blob/main/Code/Multimodal_BERT_VCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Apr 30 23:41:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!git clone https://github.com/ritzdevp/team-samosa-tvqa

Cloning into 'team-samosa-tvqa'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 274 (delta 11), reused 0 (delta 0), pack-reused 252
Receiving objects: 100% (274/274), 2.76 MiB | 14.49 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [3]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc" -O tvqa_imagenet_resnet101_pool5_hq.tar.gz && rm -rf /tmp/cookies.txt

--2022-04-30 23:43:22--  https://docs.google.com/uc?export=download&confirm=t&id=1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc
Resolving docs.google.com (docs.google.com)... 74.125.142.100, 74.125.142.139, 74.125.142.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-6g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9ffl8dnk460lg5mdiarjl1mer9ojl90f/1651362150000/07218467666063224277/*/1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc?e=download [following]
--2022-04-30 23:43:22--  https://doc-0k-6g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9ffl8dnk460lg5mdiarjl1mer9ojl90f/1651362150000/07218467666063224277/*/1m8bC4lefQsP2tRhMLAaiy0AVuBXZtegc?e=download
Resolving doc-0k-6g-docs.googleusercontent.com (doc-0k-6g-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-0k-6g-docs.googleusercontent.com (doc-0k-6g

In [ ]:
# ! tar xzf /content/tvqa_imagenet_resnet101_pool5_hq.tar.gz

^C


In [5]:
!pip install -q transformers datasets

     |████████████████████████████████| 4.0 MB 9.0 MB/s 
     |████████████████████████████████| 325 kB 47.3 MB/s 
     |████████████████████████████████| 895 kB 43.6 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 212 kB 44.9 MB/s 
     |████████████████████████████████| 136 kB 56.9 MB/s 
     |████████████████████████████████| 1.1 MB 54.1 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
     |████████████████████████████████| 94 kB 27 kB/s 
     |████████████████████████████████| 271 kB 50.4 MB/s 
     |████████████████████████████████| 144 kB 53.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [21]:
from transformers import BertTokenizer, BertModel
import torch
import transformers
from transformers import BertTokenizer, VisualBertForQuestionAnswering, VisualBertModel

In [22]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",  output_hidden_states = True)
bert_model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre",  output_hidden_states = True)


Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
% cd /content/team-samosa-tvqa/baseline_repo

/content/team-samosa-tvqa/baseline_repo


In [12]:
! chmod 777 /content/team-samosa-tvqa/baseline_repo/setup.sh

In [1]:
! /content/team-samosa-tvqa/baseline_repo/setup.sh

--2022-05-01 00:07:55--  https://tvqa.cs.unc.edu/files/tvqa_qa_release.tar.gz
Resolving tvqa.cs.unc.edu (tvqa.cs.unc.edu)... 152.2.132.230
Connecting to tvqa.cs.unc.edu (tvqa.cs.unc.edu)|152.2.132.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14474003 (14M) [application/x-gzip]
Saving to: ‘/content/tvqa_qa_release.tar.gz.1’

tvqa_qa_release.tar 100%[===================>]  13.80M  17.8MB/s    in 0.8s    

2022-05-01 00:07:56 (17.8 MB/s) - ‘/content/tvqa_qa_release.tar.gz.1’ saved [14474003/14474003]

--2022-05-01 00:07:56--  https://tvqa.cs.unc.edu/files/tvqa_subtitles.tar.gz
Resolving tvqa.cs.unc.edu (tvqa.cs.unc.edu)... 152.2.132.230
Connecting to tvqa.cs.unc.edu (tvqa.cs.unc.edu)|152.2.132.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15495443 (15M) [application/x-gzip]
Saving to: ‘/content/tvqa_subtitles.tar.gz’

tvqa_subtitles.tar. 100%[===================>]  14.78M  18.8MB/s    in 0.8s    

2022-05-01 00:08:00 (18.8 M

In [2]:
!wget http://tvqa.cs.unc.edu/files/det_visual_concepts_hq.pickle.tar.gz
!tar -xvf det_visual_concepts_hq.pickle.tar.gz

--2022-05-01 00:11:16--  http://tvqa.cs.unc.edu/files/det_visual_concepts_hq.pickle.tar.gz
Resolving tvqa.cs.unc.edu (tvqa.cs.unc.edu)... 152.2.132.230
Connecting to tvqa.cs.unc.edu (tvqa.cs.unc.edu)|152.2.132.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100718745 (96M) [application/x-gzip]
Saving to: ‘det_visual_concepts_hq.pickle.tar.gz’

det_visual_concepts 100%[===================>]  96.05M  26.1MB/s    in 5.9s    

2022-05-01 00:11:25 (16.2 MB/s) - ‘det_visual_concepts_hq.pickle.tar.gz’ saved [100718745/100718745]

det_visual_concepts_hq.pickle


In [4]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


# VCL FILE

In [64]:
import pickle
import json
import os
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwordslist = stopwords.words('english')
stopwordslist.append("?")
exceptlist = ["what", "who", "whom", "how", "where", "why"]
for w in exceptlist:
  stopwordslist.remove(w)

vis_concepts = pickle.load(open('/content/det_visual_concepts_hq.pickle', 'rb'))

embedding_index = {}
f = open('/content/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

def get_detected_objects(episode, index):
  x = vis_concepts[episode][index].strip().split(",")
  y = []
  for w in x:
    y.append(w.strip())

  return y

def phrase_splitter(arr):
  new_arr = []
  for w in arr:
    temp = w.split()
    for t in temp:
      if (t not in new_arr):
        new_arr.append(t)
  return new_arr

def get_glove_wordlist_embedding(arr_of_words):
  glove_wordlist_embedding = []
  for w in arr_of_words:
    if (w in embedding_index):
      glove_wordlist_embedding.append(embedding_index[w])
  glove_wordlist_embedding = np.array(glove_wordlist_embedding)
  return glove_wordlist_embedding


def tokenize_question(question):
  words = nltk.word_tokenize(question)
  filtered_words = [word.lower() for word in words if word.lower() not in stopwordslist ]
  qarr = []
  for w in filtered_words:
    if w in embedding_index:
      qarr.append(w)
  return qarr

def relevance_score(episode, frame_index, question):
  #function to get relevance score between question and detected objects in a frame
  detected_objects_list = get_detected_objects(episode, frame_index)[:10]
  words = nltk.word_tokenize(question)

  #to split words like "pink shirt" to "pink" and "shirt"
  detected_objects_list = phrase_splitter(detected_objects_list)

  #num_det is number of words in detected objects list
  #num_q is number of words in question
  det_objs_embedding_matrix = get_glove_wordlist_embedding(detected_objects_list) #num_det x 50
  question_embedding_matrix = get_glove_wordlist_embedding(tokenize_question(question)) #num_q x 50

  if (len(detected_objects_list) == 0):
    return np.zeros(10), 0

  score_matrix = det_objs_embedding_matrix.dot(question_embedding_matrix.T)  #num_det x num_q
  score = np.sum(score_matrix)

  return score_matrix, score

### returns maximum sum subarray of given window size

def get_maxsum_subarray(myarr, window_size):
  start = 0
  sum = 0
  currlen = 0
  ws = window_size
  maxsum = 0
  maxindexend = -1
  for i in range(len(myarr) - ws):
    sum += myarr[i]
    currlen += 1
    if (currlen > ws):
      sum -= myarr[i - ws]
      currlen -= 1
    
    if (sum > maxsum):
      maxsum = sum
      maxindexend = i
  
  maxstartindex = maxindexend - ws + 1
  return maxstartindex, maxsum


### returns relevance scores of all frames for the given question
def get_relevance_scorelist(vid_name, question):
  num_frames = len(vis_concepts[vid_name])
  score_list = np.zeros(num_frames)

  max_score_index = -1
  max_score = 0
  min_score_index = -1
  min_score = 10000

  for frame_index in range(num_frames):
    score_matrix, score = relevance_score(vid_name, frame_index, question)
    score_list[frame_index] = score
    if (score > max_score):
      max_score = score
      max_score_index = frame_index
    if (score < min_score):
      min_score = score
      min_score_index = frame_index

  return score_list, max_score_index, min_score_index

def make_link(vid_name, frame_index):
  dummy = "https://tvqammml.s3.us-east-1.amazonaws.com/"
  temp = "00000"
  temp = (temp + str(frame_index))[-5:]
  url = dummy + vid_name + '/' + temp + '.jpg'
  return url

def plot_image_from_url(url):
  image = io.imread(url)
  plt.imshow(image)
  plt.show()

def plot_max_relevance_frames(vid_name, start_index, window_size):
  dummy = "https://tvqammml.s3.us-east-1.amazonaws.com/"
  fig = plt.figure(figsize=(10, 20))
  columns = 2
  rows = window_size/2
  j = start_index
  for i in range(1, window_size +1):
      temp = "00000"
      temp = (temp + str(j))[-5:]
      url = dummy + vid_name + '/' + temp + '.jpg'
      img = io.imread(url)
      fig.add_subplot(rows, columns, i)
      plt.imshow(img)
      j += 1
  plt.show()


"""
Returns the unique objects detected in an episode from
a start frame index, given the window size
"""

def get_unique_objs_in_chunk(episode, start_index, window_size):
  num_frames = len(vis_concepts[episode])
  temp = set()
  for i in range(start_index, start_index + window_size, 1):
    arr = get_detected_objects(episode, i)
    arr = phrase_splitter(arr)
    for w in arr:
      temp.add(w)
  return list(temp)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
#debugging
import h5py
import torch
from torch.nn.utils.rnn import pad_sequence
from CONSTANTS import RESNET_FEATURES

h5driver=None
vid_feat_path=RESNET_FEATURES
vid_h5 = h5py.File(vid_feat_path, "r", driver=h5driver)

def read_resnet_feats(video_names, questions, stride=8, VCL_window_size=10):
    video_resnet_feat = []
    unique_objects_per_vid = []
    assert len(video_names) == len(questions)
    for i, video in enumerate(video_names):
        quest_i = questions[i]
        score_list, max_score_index, min_score_index = get_relevance_scorelist(video, quest_i)
        start_index, max_score = get_maxsum_subarray(score_list, VCL_window_size)
        unique_objects = get_unique_objs_in_chunk(video, start_index, VCL_window_size)
        
        unique_objects_per_vid.append(' '.join(unique_objects))
        video_resnet_feat.append(torch.tensor(vid_h5[video][start_index:start_index+VCL_window_size, :], device="cuda"))

    video_resnet_feat =  pad_sequence(video_resnet_feat, batch_first=True)
    
    return video_resnet_feat, unique_objects_per_vid

In [66]:
class TVQA_Multimodal(torch.nn.Module):

    def __init__(self):

        super(TVQA_Multimodal, self).__init__()

        self.bert_tokenizer = bert_tokenizer
        self.bert_model = bert_model.to("cuda")

        self.proj = torch.nn.Sequential(torch.nn.Linear(768, 256),
                                        torch.nn.GELU(),
                                        torch.nn.Linear(256, 64),
                                        torch.nn.GELU(),
                                        torch.nn.Linear(64, 1)).to("cuda")

    def _freeze_bert_weights(self, requires_grad_layers = ['encoder.layer.11',
                                                           'pooler.dense.weight',
                                                           'pooler.dense.bias']):
        for name, p in self.bert_model.named_parameters():
            req_grad = False
            for layer_name in requires_grad_layers:
                if layer_name in name:
                    req_grad = True
                    break

            p.requires_grad = req_grad
        
        return True

    def merge_q_ans(self, que, ans):
        merged = []
        for i in range(len(que)):
          curr = que[i]+ ' [SEP] ' + ans[i]
          merged.append(curr)
        return merged

    def merge_subt_visconcepts(self, subtitles, unique_objects):
        merged = []
        for i in range(len(subtitles)):
          curr = subtitles[i]+ ' [SEP] ' + unique_objects[i]
          merged.append(curr)
        return merged


    def bert_forward(self, question, ans, subt_text, video_names):
        #all arguments have batch size
        visual_embeds, unique_objs_per_vid = read_resnet_feats(video_names, question)
        merge_q_ans = self.merge_q_ans(question, ans)
        merge_sub_vc = self.merge_subt_visconcepts(subt_text, unique_objs_per_vid)

        # print("merge sub vc", merge_sub_vc)
        # print("merge sub vc 0", merge_sub_vc[0])
        

        inputs       = bert_tokenizer(merge_sub_vc, 
                                      merge_q_ans,  
                                      padding="max_length",
                                      truncation=True,
                                      return_token_type_ids=True,
                                      return_attention_mask=True,
                                      add_special_tokens=True,
                                      return_tensors="pt")
        
        

        # print("---subt text --", len(subt_text))
        # print("---unique_objs_per_vid--", len(unique_objs_per_vid))
        # print('vis embed shape', visual_embeds.shape)
        # print('unique obj', unique_objs_per_vid)
        # print("--subt--", subt_text)
        

        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long).to("cuda")
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float).to("cuda")
        inputs.update(
            {
                "visual_embeds": visual_embeds,
                "visual_token_type_ids": visual_token_type_ids,
                "visual_attention_mask": visual_attention_mask,
            }
        )
        inputs = inputs.to("cuda")

        output = bert_model(**inputs)

        hidden_states = output.last_hidden_state
        cls_tokens = hidden_states[:,0,:]

        return cls_tokens

    def forward(self, question, subt_text, a0, a1, a2, a3, a4, video_names):

        bert_a0 = self.bert_forward(question=question,
                                    ans=a0,
                                    subt_text=subt_text,
                                    video_names=video_names)        
        score_a0 = self.proj(bert_a0)

        bert_a1 = self.bert_forward(question=question,
                                    ans=a1,
                                    subt_text=subt_text,
                                    video_names=video_names)
        score_a1 = self.proj(bert_a1)

        bert_a2 = self.bert_forward(question=question,
                                    ans=a2,
                                    subt_text=subt_text,
                                    video_names=video_names)
        score_a2 = self.proj(bert_a2)

        bert_a3 = self.bert_forward(question=question,
                                    ans=a3,
                                    subt_text=subt_text,
                                    video_names=video_names)
        
        score_a3 = self.proj(bert_a3)

        bert_a4 = self.bert_forward(question=question,
                                    ans=a4,
                                    subt_text=subt_text,
                                    video_names=video_names)
        
        score_a4 = self.proj(bert_a4)

        # print(" score_a4 ", score_a4.shape)


        logits = torch.cat((score_a0, score_a1, score_a2, score_a3, score_a4), dim=1)

        # print(" logits ", logits.shape)


        return logits


In [67]:
from CONSTANTS import  BASE_PATH
import json

In [68]:


class TVQAPlus(torch.utils.data.Dataset):
    def __init__(self, dataset='train'):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        TRAIN_DICT_JSON = BASE_PATH + "/tvqa_plus/tvqa_plus_train.json"
        VAL_DICT_JSON = BASE_PATH + '/tvqa_plus/tvqa_plus_val.json'
        SUBTITLES_DICT_JSON = BASE_PATH + '/tvqa_plus/tvqa_plus_subtitles.json'

        self.dataset = dataset

        train_dict = {}
        val_dict = {}
        subtitles_dict = {}

        with open(TRAIN_DICT_JSON) as f:
          train_dict = json.load(f)

        with open(VAL_DICT_JSON) as f:
          val_dict = json.load(f)

        with open(SUBTITLES_DICT_JSON) as f:
          self.subtitles_dict = json.load(f)

        self.target_dict = {}
        if self.dataset == 'train':
          self.target_dict = train_dict
        elif self.dataset == 'val':
          self.target_dict = val_dict
        else:
          raise Exception(f"Invalid dataset passed {self.dataset}")
      
    def __len__(self):
        return len(self.target_dict)
        
    def __getitem__(self, i):

      question = self.target_dict[i]['q']
    
      a0       = self.target_dict[i]['a0']
      a1       = self.target_dict[i]['a1']
      a2       = self.target_dict[i]['a2']
      a3       = self.target_dict[i]['a3']
      a4       = self.target_dict[i]['a4']

      answer_idx = int(self.target_dict[i]['answer_idx'])
      
      video_name = self.target_dict[i]['vid_name']
      subtitle = self.subtitles_dict[video_name]
      subt_text = subtitle['sub_text']

      return question, subt_text, a0, a1, a2, a3, a4, video_name, answer_idx

In [69]:
train_dataset = TVQAPlus('train')
test_dataset = TVQAPlus("val")

In [70]:
tvqa_model = TVQA_Multimodal()
tvqa_model._freeze_bert_weights()

True

In [71]:
batch_size=24
dev_batch_size=12

In [72]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=dev_batch_size, shuffle=False)

In [73]:
# for batch_idx, ( question, subt_text, a0, a1, a2, a3, a4, video_names, answer_idx) in enumerate(train_loader):
#     logits = tvqa_model.forward(question, subt_text, a0, a1, a2, a3, a4, video_names)
#     break

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [74]:
def merge(que, ans):
  merged = []
  for i in range(len(que)):
    curr = que[i]+ ' [SEP] ' + ans[i]
    merged.append(curr)
  return merged

In [75]:
# tvqa_model = TVQA_Multimodal()
# tvqa_model._freeze_bert_weights()

print(tvqa_model.named_parameters())

for name, p in tvqa_model.named_parameters():
    print("name", name, "p", p.requires_grad)

<generator object Module.named_parameters at 0x7f7afde31b50>
name bert_model.embeddings.word_embeddings.weight p False
name bert_model.embeddings.position_embeddings.weight p False
name bert_model.embeddings.token_type_embeddings.weight p False
name bert_model.embeddings.LayerNorm.weight p False
name bert_model.embeddings.LayerNorm.bias p False
name bert_model.embeddings.visual_token_type_embeddings.weight p False
name bert_model.embeddings.visual_position_embeddings.weight p False
name bert_model.embeddings.visual_projection.weight p False
name bert_model.embeddings.visual_projection.bias p False
name bert_model.encoder.layer.0.attention.self.query.weight p False
name bert_model.encoder.layer.0.attention.self.query.bias p False
name bert_model.encoder.layer.0.attention.self.key.weight p False
name bert_model.encoder.layer.0.attention.self.key.bias p False
name bert_model.encoder.layer.0.attention.self.value.weight p False
name bert_model.encoder.layer.0.attention.self.value.bias p Fal

In [76]:
def val_acc(model, dev_loader, batch_size_dev):
  model.eval()
  num_correct = 0
  for batch_idx, ( question, subt_text, a0, a1, a2, a3, a4, video_names, answer_idx) in enumerate(test_loader):
    answer_idx = answer_idx.to("cuda")
    with torch.no_grad():
      # # IF MODEL does not TAKES VIDEO INPUT

      logits = model.forward(question, subt_text, a0, a1, a2, a3, a4, video_names)

      
    num_correct += int((torch.argmax(logits, axis=1) == answer_idx).sum())
    acc = 100 * num_correct / ((batch_idx + 1) * batch_size_dev)

  dev_acc = 100 * num_correct / (len(dev_loader) * batch_size_dev)

  model.train()
  return dev_acc

In [77]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

optimizer = optim.Adam(tvqa_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [80]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
torch.cuda.empty_cache()

# Set the verbosity level as follows:

transformers.logging.set_verbosity_error()


# bert_model.cuda()

model_version="VCL_model.pth"


epoch = 0
best_dev_acc = 0
loss_epoch = 0

while epoch < 100:
  
    num_correct = 0

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    tvqa_model.train()


    if os.path.exists(f'/content/drive/MyDrive/MMML/{model_version}'):
        # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

        checkpoint = torch.load(f'/content/drive/MyDrive/MMML/{model_version}')
        tvqa_model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch'] + 1

    for batch_idx, ( question, subt_text, a0, a1, a2, a3, a4, video_names, answer_idx) in enumerate(train_loader):

        logits = tvqa_model.forward(question, subt_text, a0, a1, a2, a3, a4, video_names)
        answer_idx = answer_idx.to("cuda")
        loss = criterion(logits, answer_idx)
        num_correct += int((torch.argmax(logits, axis=1) == answer_idx).sum())

        loss.backward()
        optimizer.step()
        loss_epoch += float(loss)
        optimizer.zero_grad()

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((batch_idx + 1) * batch_size)),
            loss="{:.04f}".format(float(loss_epoch / (batch_idx + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() # Update tqdm bar

        # break
        torch.cuda.empty_cache()


    batch_bar.close() # You need this to close the tqdm bar

    train_acc = 100 * num_correct / (len(train_loader) * batch_size)
    dev_acc = val_acc(tvqa_model, test_loader, dev_batch_size)

    print(f'Epoch {epoch} Loss {loss_epoch} train_acc {train_acc}, devacc {dev_acc}')

    torch.save({
            'epoch': epoch,
            'model_state_dict': tvqa_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_epoch/len(train_loader),
            },  f'/content/drive/MyDrive/MMML/{model_version}')
    
    if dev_acc > best_dev_acc:
        best_dev_acc = dev_acc

        torch.save({
                'epoch': epoch,
                'model_state_dict': tvqa_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss_epoch/len(train_loader),
                },  f'/content/drive/MyDrive/MMML/best_dev_acc_{model_version}')

        


Train:   1%|          | 10/982 [01:58<3:10:49, 11.78s/it, acc=34.1667%, loss=1.5304, lr=0.0010, num_correct=82]

In [ ]:
torch.save({
      'epoch': epoch,
      'model_state_dict': tvqa_model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss_epoch/len(train_loader),
      },  f'/content/drive/MyDrive/MMML/bert_backprop.pth')

In [ ]:
tvqa_model = TVQA_Multimodal()
tvqa_model._freeze_bert_weights()
# print(tvqa_model)
# tvqa_model._free

for name, p in tvqa_model.bert_model.named_parameters():
    print("name", name, "p", p.shape, "p.requires_grad", p.requires_grad)